In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from torchlego import *
import torch.nn as nn
from torchsummary import summary

In [3]:
import torch.nn as nn
import torch
from torchvision.models import resnet34
from torchlego import *
from torchsummary import summary
from torchlego.utils import ModuleTransfer

def resnet_basic_block(in_features, out_features):
    shortbut = conv_bn(in_features, out_features, kernel_size=1, stride=2, bias=False) if in_features != out_features else nn.Identity()
    stride = 2 if in_features != out_features else 1
    return nn.Sequential(
                Add(nn.ModuleList([
                    nn.Sequential(
                        conv3x3_bn_act(in_features, out_features, stride=stride, bias=False),
                        conv3x3_bn(out_features, out_features, bias=False))]), 
                    shortcut=shortbut),
                nn.ReLU())
ResNet34 = lambda : nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    nn.Sequential(*[resnet_basic_block(64, 64)] * 3),
    resnet_basic_block(64, 128),
    nn.Sequential(*[resnet_basic_block(128, 128)] * 3),
    resnet_basic_block(128, 256),
    nn.Sequential(*[resnet_basic_block(256, 256)] * 5),
    resnet_basic_block(256, 512),
    nn.Sequential(*[resnet_basic_block(512, 512)] * 2),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(512, 1000)
)

x = torch.rand((1,3,224,244))

resnet34_pre = resnet34(True)

module_stranfer = ModuleTransfer(resnet34_pre, ResNet34())
resnet_trans = module_stranfer(x)

In [10]:
resnet34_pre

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
resnet_trans

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Sequential(
      (0): Residual(
        (blocks): ModuleList(
          (0): Sequential(
            (0): Sequential(
              (0): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
            (1): Sequential(
              (0): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
        )
        (shortcut): Identity()
      )
      (1): ReLU()
    )
    (1):

### Unet

In [ ]:
# down = lambda in_features, out_features: nn.Sequential(
#     conv3x3(in_features, in_features),
#     conv3x3(in_features, out_features),
#     nn.MaxPool2d(out_features)
# )

# up = lambda in_features, out_features : nn.Sequential(
#     conv3x3(in_features, in_features),
#     conv3x3(in_features, out_features),
#     nn.ConvTranspose2d(out_features, out_features, kernel_size=2)
# )

# Concat(
#     [
#         down(64, 128),
#         down(128, 256),
#         down(256, 512),
#         down(512, 1024),
#     ],
#     [
#         up(1024, 512 * 2),
#         up(256 * 2, 128),
#         up(128 * 2, 64),
#         nn.Sequential(
#             conv3x3(64), 
#             conv3x3(64))
#     ]
# )